construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**


#Import Libraries

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


# Load in Data


In [2]:
# open text file and read in data as text
with open('text.txt','r') as f:
  text = f.read()

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

#Tokenization
In the cells, below, I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# we create two dictionaries:
# 1. int2char, which maps integers to chars
# 2. chat2int, which maps chars to integers

chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# endcoed the text
encoded = np.array([char2int[ch] for ch in text])

In [5]:
encoded[:100]

array([67, 41, 30, 52, 64, 37, 65, 73, 50, 81, 81, 81, 56, 30, 52, 52, 39,
       73, 74, 30, 24, 44, 35, 44, 37, 36, 73, 30, 65, 37, 73, 30, 35, 35,
       73, 30, 35, 44, 33, 37, 20, 73, 37, 13, 37, 65, 39, 73, 48, 68, 41,
       30, 52, 52, 39, 73, 74, 30, 24, 44, 35, 39, 73, 44, 36, 73, 48, 68,
       41, 30, 52, 52, 39, 73, 44, 68, 73, 44, 64, 36, 73, 62, 46, 68, 81,
       46, 30, 39, 32, 81, 81, 31, 13, 37, 65, 39, 64, 41, 44, 68])

# Pre-processing the data

In [8]:
def one_hot_encode(arr, n_labels):
  
  # Initializing the encoded array
  one_hot = np.zeros( (arr.size, n_labels), dtype = np.float32 )

  # Fill the appropirate elements with ones
  one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1


  one_hot = one_hot.reshape((*arr.shape, n_labels))

  return one_hot


In [9]:
test_seq = np.array([[3,5,1]])
one_hot = one_hot_encode(test_seq,8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


# Making training mini-batches


In [10]:
def get_batches(arr, batch_size , seq_length):

  '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
  batch_size_total = batch_size * seq_length
  # total number of batches we can make
  n_batches = len(arr)//batch_size_total

  # Keep only enough characters to make full batches
  arr = arr[:n_batches * batch_size_total]
 
  # Reshape into batch_size rows
  arr = arr.reshape((batch_size,-1))

  # iterate through the array, one sequence at a time
  for n in range(0, arr.shape[1], seq_length):
    
    x = arr[:, n:n+seq_length]
    # the targets, shifted by one
    y = np.zeros_like(x)
    try:
      y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
    except IndexError:
      y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
    yield x, y


# Test

In [11]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [12]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[67 41 30 52 64 37 65 73 50 81]
 [36 62 68 73 64 41 30 64 73 30]
 [37 68 71 73 62 65 73 30 73 74]
 [36 73 64 41 37 73 57 41 44 37]
 [73 36 30 46 73 41 37 65 73 64]
 [57 48 36 36 44 62 68 73 30 68]
 [73  8 68 68 30 73 41 30 71 73]
 [45 21 35 62 68 36 33 39 32 73]]

y
 [[41 30 52 64 37 65 73 50 81 81]
 [62 68 73 64 41 30 64 73 30 64]
 [68 71 73 62 65 73 30 73 74 62]
 [73 64 41 37 73 57 41 44 37 74]
 [36 30 46 73 41 37 65 73 64 37]
 [48 36 36 44 62 68 73 30 68 71]
 [ 8 68 68 30 73 41 30 71 73 36]
 [21 35 62 68 36 33 39 32 73 17]]


# Defining the network with PyTorch
### Model Structure
In __init__ the suggested structure is as follows:

* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size n_hidden, a number of layers n_layers, a dropout probability drop_prob, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with drop_prob
* Define a fully-connected layer with params: input size n_hidden and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the __init__ function, and we use them and store them by doing something like self.drop_prob = drop_prob.




In [13]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [14]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        
        self.dropout = nn.Dropout(drop_prob)
        
        # fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        # Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        # pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        # put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

# Train

In [15]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

# Instantiating the model
Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes, and start training!

In [16]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [17]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)


Epoch: 1/20... Step: 10... Loss: 3.2829... Val Loss: 3.2305
Epoch: 1/20... Step: 20... Loss: 3.1553... Val Loss: 3.1431
Epoch: 1/20... Step: 30... Loss: 3.1486... Val Loss: 3.1257
Epoch: 1/20... Step: 40... Loss: 3.1162... Val Loss: 3.1197
Epoch: 1/20... Step: 50... Loss: 3.1445... Val Loss: 3.1179
Epoch: 1/20... Step: 60... Loss: 3.1175... Val Loss: 3.1161
Epoch: 1/20... Step: 70... Loss: 3.1098... Val Loss: 3.1151
Epoch: 1/20... Step: 80... Loss: 3.1239... Val Loss: 3.1124
Epoch: 1/20... Step: 90... Loss: 3.1244... Val Loss: 3.1059
Epoch: 1/20... Step: 100... Loss: 3.1019... Val Loss: 3.0927
Epoch: 1/20... Step: 110... Loss: 3.0755... Val Loss: 3.0656
Epoch: 1/20... Step: 120... Loss: 2.9909... Val Loss: 2.9899
Epoch: 1/20... Step: 130... Loss: 2.9549... Val Loss: 2.9188
Epoch: 2/20... Step: 140... Loss: 2.8210... Val Loss: 2.8135
Epoch: 2/20... Step: 150... Loss: 2.7311... Val Loss: 2.6861
Epoch: 2/20... Step: 160... Loss: 2.6219... Val Loss: 2.5829
Epoch: 2/20... Step: 170... Loss:

# Checkpoint

In [18]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

# Making Predictions


In [19]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

# generating text

In [20]:
def sample(net, size, prime='The', top_k=None):
        
    global char
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [21]:
print(sample(net, 1000, prime='Anna', top_k=5))


Anna, who was
and for his face. But to say something about the mononce of his
hind home, watched the cape, and almost taking his his wife, the
prince.

"Yes, you're not letting it."

"Anna has so such a side--as I'll go awina and most asking only a life
which I always begin to be done; and I don't call it, then you want
all sometomenter, that it wants to say, that.... You must have
been the same."

The crush of this comparions had already set about him. But there were
the fact would not cannot have been bound."

At times, the soul they could not help have brother that it
seemed to her, he saw the condution of the forest. He saw that
seemed to him how the chears, to those wend with the propiritic of the step
which he clutched his hands, and there will say to her heart, as had
the study.

"That manners of some point of an instant, or the carriage as a
solled crowd and sense, and that you were a strange."

And was a smile, and to tell her a thought, but he had saw a condliant
daughter and

# Loading a checkpoint


In [22]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))


And Levin said
such a minute.

At that importance of the strange with his forest, all the common
people showed without hurriedly and the position of the pronding house,
their son, she went on women, before she felt her for the most
childled and husing. He felt that this came out and to hold that
starting to the carriage of his house, had been to be a soul of the
single of the men, and with the children had been bound to anyone, and
at the draw to her fire helpers, and the first part world and watch had
been all his settened and his face. The same well as if they was an one
of the moments and sincerely should give it to him to be done on all the
door.

"Well, we shall go out."

"You devermonate my wife, and there would be a stair,
and when she has a servant, but in this carriage is themselves. If you
want to say something, but it's all so all three morning to him.
And the painter of myself almust must stay for a secret most
colonel."

"I don't know it with the marsh."

"I say to her, an